IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M5\FAULT_M5_10


In [3]:
guasto=2

VOLO m5 FAULT 10%

In [4]:
rcou_m5_fault10 = pd.read_csv("RCOU.csv")
rcou_m5_fault10 = rcou_m5_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m5_fault10 = rcou_m5_fault10[((rcou_m5_fault10['C9'] >= potenza) & (rcou_m5_fault10['C10'] >= potenza) & (rcou_m5_fault10['C11'] >= potenza) & (rcou_m5_fault10['C12'] >= potenza) & (rcou_m5_fault10['C13'] >= potenza) & (rcou_m5_fault10['C14']>= potenza))]
rcou_m5_fault10=rcou_m5_fault10.reset_index(drop=True)
display(rcou_m5_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,72720687,1542,1391,1459,1484,1528,1408
1,72731556,1415,1540,1494,1466,1428,1529
2,72741603,1472,1506,1414,1556,1393,1572
3,72752252,1524,1471,1426,1562,1454,1539
4,72762572,1495,1518,1476,1535,1461,1549
...,...,...,...,...,...,...,...
7390,166795551,1497,1409,1482,1427,1398,1507
7391,166805911,1488,1417,1464,1443,1369,1527
7392,166817138,1469,1437,1481,1424,1367,1528
7393,166827779,1454,1450,1494,1407,1369,1525


In [5]:
min=rcou_m5_fault10['TimeUS'][0]
max=rcou_m5_fault10['TimeUS'][len(rcou_m5_fault10)-1]
print(max)
print(min)

166914844
72720687


In [6]:
xkf1_m5_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m5_fault10 = xkf1_m5_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_fault10 = xkf1_m5_fault10[((xkf1_m5_fault10['TimeUS'] >= min) & (xkf1_m5_fault10['TimeUS'] <= max	))]
xkf1_m5_fault10 = xkf1_m5_fault10.reset_index(drop=True)
print(xkf1_m5_fault10)

         TimeUS  Roll  Pitch   Yaw
0      72729675 -0.65   0.55  2.33
1      72739678 -0.66   0.65  2.33
2      72750119 -0.61   0.61  2.33
3      72760283 -0.65   0.60  2.34
4      72770308 -0.62   0.57  2.37
...         ...   ...    ...   ...
8533  166869222  1.82  -2.80  4.06
8534  166879639  1.70  -2.83  4.07
8535  166890212  1.68  -2.82  4.08
8536  166900589  1.69  -2.85  4.07
8537  166912734  1.77  -2.79  4.08

[8538 rows x 4 columns]


In [7]:
att_m5_fault10 = pd.read_csv("ATT.csv")
att_m5_fault10 = att_m5_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_fault10 = att_m5_fault10[((att_m5_fault10['TimeUS'] >= min) & (att_m5_fault10['TimeUS'] <= max	))]
att_m5_fault10=att_m5_fault10.reset_index(drop=True)
print(att_m5_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      72730714    -0.21 -0.65     -0.06   0.55    2.27  2.33
1      72740788    -0.22 -0.66     -0.06   0.65    2.27  2.33
2      72751532    -0.23 -0.61     -0.07   0.61    2.27  2.33
3      72761836    -0.22 -0.65     -0.07   0.60    2.27  2.34
4      72772125    -0.21 -0.62     -0.09   0.57    2.27  2.37
...         ...      ...   ...       ...    ...     ...   ...
8534  166870527     1.97  1.82      0.13  -2.80    5.21  4.06
8535  166881106     1.95  1.70      0.12  -2.83    5.21  4.07
8536  166891684     1.92  1.68      0.11  -2.82    5.21  4.08
8537  166902181     1.88  1.69      0.11  -2.85    5.21  4.07
8538  166914063     1.84  1.77      0.11  -2.79    5.21  4.08

[8539 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_fault10)>len(xkf1_m5_fault10):
    to_add=att_m5_fault10[len(xkf1_m5_fault10):]
    att_m5_fault10=att_m5_fault10[:len(xkf1_m5_fault10)]

if len(xkf1_m5_fault10)>len(att_m5_fault10):
    to_add=xkf1_m5_fault10[len(att_m5_fault10):]

for idx,i in enumerate(att_m5_fault10['Roll']):
    if(xkf1_m5_fault10['Roll'][idx] != i):
        att_m5_fault10['Roll'][idx] = ((att_m5_fault10['Roll'][idx] + xkf1_m5_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m5_fault10['Pitch']):
    if(xkf1_m5_fault10['Pitch'][idx] != i):
        att_m5_fault10['Pitch'][idx] = ((att_m5_fault10['Pitch'][idx] + xkf1_m5_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_fault10['Yaw']):
    if(xkf1_m5_fault10['Yaw'][idx] != i):
        att_m5_fault10['Yaw'][idx] = ((att_m5_fault10['Yaw'][idx] + xkf1_m5_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_fault10=pd.concat([att_m5_fault10,to_add])

print(att_m5_fault10)

C:\Users\mgale\AppData\Local\Temp\ipykernel_12808\4076637029.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m5_fault10['Roll'][idx] = ((att_m5_fault10['Roll'][idx] + xkf1_m5_fault10['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12808\4076637029.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m5_fault10['Pitch'][idx] = ((att_m5_fault10['Pitch'][idx] + xkf1_m5_fault10['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12808\4076637029.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw
0      72730714    -0.21 -0.650     -0.06  0.550    2.27  2.330
1      72740788    -0.22 -0.660     -0.06  0.650    2.27  2.330
2      72751532    -0.23 -0.610     -0.07  0.610    2.27  2.330
3      72761836    -0.22 -0.650     -0.07  0.600    2.27  2.340
4      72772125    -0.21 -0.620     -0.09  0.570    2.27  2.370
...         ...      ...    ...       ...    ...     ...    ...
8534  166870527     1.97  1.760      0.13 -2.815    5.21  4.065
8535  166881106     1.95  1.690      0.12 -2.825    5.21  4.075
8536  166891684     1.92  1.685      0.11 -2.835    5.21  4.075
8537  166902181     1.88  1.730      0.11 -2.820    5.21  4.075
8538  166914063     1.84  1.770      0.11 -2.790    5.21  4.080

[8539 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m5_fault10 = esc_0_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_fault10 = esc_0_m5_fault10[((esc_0_m5_fault10['TimeUS'] >= min) & (esc_0_m5_fault10['TimeUS'] <= max))]
esc_0_m5_fault10=esc_0_m5_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m5_fault10 = esc_1_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_fault10 = esc_1_m5_fault10[((esc_1_m5_fault10['TimeUS'] >= min) & (esc_1_m5_fault10['TimeUS'] <= max))]
esc_1_m5_fault10=esc_1_m5_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m5_fault10 = esc_2_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_fault10 = esc_2_m5_fault10[((esc_2_m5_fault10['TimeUS'] >= min) & (esc_2_m5_fault10['TimeUS'] <= max))]
esc_2_m5_fault10=esc_2_m5_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m5_fault10 = esc_3_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_fault10 = esc_3_m5_fault10[((esc_3_m5_fault10['TimeUS'] >= min) & (esc_3_m5_fault10['TimeUS'] <= max))]
esc_3_m5_fault10=esc_3_m5_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m5_fault10 = esc_4_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_fault10 = esc_4_m5_fault10[((esc_4_m5_fault10['TimeUS'] >= min) & (esc_4_m5_fault10['TimeUS'] <= max))]
esc_4_m5_fault10=esc_4_m5_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m5_fault10 = esc_5_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_fault10 = esc_5_m5_fault10[((esc_5_m5_fault10['TimeUS'] >= min) & (esc_5_m5_fault10['TimeUS'] <= max))]
esc_5_m5_fault10=esc_5_m5_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m5_fault10 = imu_0_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_fault10 = imu_0_m5_fault10[((imu_0_m5_fault10['TimeUS'] >= min) & (imu_0_m5_fault10['TimeUS'] <= max))]

imu_1_m5_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m5_fault10 = imu_1_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_fault10 = imu_1_m5_fault10[((imu_1_m5_fault10['TimeUS'] >= min) & (imu_1_m5_fault10['TimeUS'] <= max))]

imu_2_m5_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m5_fault10 = imu_2_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_fault10 = imu_2_m5_fault10[((imu_2_m5_fault10['TimeUS'] >= min) & (imu_2_m5_fault10['TimeUS'] <= max))]

imu_m5_fault10 = pd.concat((imu_0_m5_fault10, imu_1_m5_fault10, imu_2_m5_fault10))
imu_m5_fault10=imu_m5_fault10.groupby(imu_m5_fault10.TimeUS, as_index=False).mean()

display(imu_m5_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,72725912,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562
1,72728103,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303
2,72730554,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179
3,72733710,0.119616,-0.211923,-0.026932,0.110713,0.545332,-9.907626
4,72735559,0.112437,-0.099032,-0.026028,-0.145339,0.406057,-10.143177
...,...,...,...,...,...,...,...
32433,166897233,0.031076,0.046551,0.022947,-0.416947,0.473361,-9.839571
32434,166899201,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236
32435,166902063,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467
32436,166904726,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_fault10 = pd.merge_ordered(imu_m5_fault10,att_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_0_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_1_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_2_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_3_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_4_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_5_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")
m5_fault10 = pd.merge_ordered(m5_fault10, rcou_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m5_fault10["TimeUS"] = m5_fault10["TimeUS"] - m5_fault10.iloc[0]["TimeUS"]
m5_fault10["TimeUS"] = m5_fault10["TimeUS"].astype(int)
m5_fault10["Guasto"] = guasto

In [13]:
last_cell = m5_fault10.iloc[-1, m5_fault10.columns.get_loc('TimeUS')]
print(last_cell)

94194157


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
32965  94181005
32966  94183862
32967  94186719
32968  94189576
32969  94192433

[32970 rows x 1 columns]


In [16]:
m5_fault10_final = pd.merge_ordered(m5_fault10,df)
m5_fault10_final=m5_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m5_fault10_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
1          2857  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
2          5225  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
3          5714  0.033646  0.092000 -0.004889 -0.370116  0.394487 -10.111562   
4          7416 -0.245029  0.217662  0.038900 -0.212036  0.573743  -9.924303   
...         ...       ...       ...       ...       ...       ...        ...   
86112  94189576  0.015743  0.110068  0.012201 -0.670795  0.566046 -10.021941   
86113  94192433  0.015743  0.110068  0.012201 -0.670795  0.566046 -10.021941   
86114  94193154 -0.053250  0.057050 -0.006003 -0.753840  0.213708 -10.067043   
86115  94193376 -0.053250  0.057050 -0.006003 -0.753840  0.213708 -10.067043   
86116  94194157 -0.053250  0.057050 -0.006003 -0.753840  0.213708 -10.067043   

       DesRoll  Roll  DesPitch  ...  Cu

In [17]:
m5_fault10_final = m5_fault10_final[m5_fault10_final.TimeUS.isin(to_be)]

In [18]:
m5_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
1,2857,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
3,5714,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
5,8571,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
9,11428,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86106,94181005,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236,1.92,1.685,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
86109,94183862,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
86111,94186719,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
86112,94189576,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0


In [19]:
from datetime import timedelta
m5_fault10_final=m5_fault10_final.reset_index(drop=True)
m5_fault10_final['TimeUS'] = pd.to_datetime(m5_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
1,00:00:00.002857,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
2,00:00:00.005714,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
3,00:00:00.008571,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2.0
4,00:00:00.011428,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32965,00:01:34.181005,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236,1.92,1.685,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
32966,00:01:34.183862,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
32967,00:01:34.186719,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0
32968,00:01:34.189576,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m5_fault10_final)/350)):
        V1=m5_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12808\212889577.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12808\212889577.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.055093,0.029366,-1.086720,0.179770,0.0,100.0,10.166016,2.788774,-0.010972,...,6448.270335,4.648120,0.326712,-0.464927,4.683033,86.0,83.0,24.883960,8.273270,2
1,1.0,-0.055163,0.029284,-1.080695,0.179565,0.0,94.0,10.428676,2.803751,-0.010751,...,6522.713332,4.651449,0.329683,-0.477396,4.686654,82.0,80.0,23.892985,8.677289,2
2,2.0,-0.055297,0.029207,-1.074590,0.179393,0.0,96.0,10.666993,2.793884,-0.010998,...,6566.463479,4.654759,0.332615,-0.490080,4.690251,86.0,84.0,22.848873,8.821958,2
3,3.0,-0.055331,0.029124,-1.068760,0.179175,0.0,95.0,10.938631,2.816814,-0.010890,...,6583.087785,4.658051,0.335508,-0.502940,4.693826,86.0,84.0,21.755326,8.714139,2
4,4.0,-0.055365,0.029043,-1.062930,0.178958,0.0,92.0,11.208624,2.837513,-0.010783,...,6576.456353,4.661324,0.338363,-0.515939,4.697377,88.0,86.0,20.616143,8.723966,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89.0,-0.052601,0.025284,-0.904218,0.167314,0.0,77.0,29.623853,2.785118,-0.016284,...,6728.359551,4.965523,0.660682,-1.121415,5.031461,120.0,119.0,22.267217,11.331422,2
90,90.0,-0.052184,0.025304,-0.912548,0.167241,0.0,86.0,29.931155,3.517187,-0.016264,...,6533.107744,4.968458,0.662980,-1.129328,5.034586,121.0,120.0,21.712271,12.100697,2
91,91.0,-0.052396,0.025266,-0.908585,0.167195,0.0,99.0,29.959503,3.674268,-0.016025,...,6314.806015,4.971380,0.665251,-1.137127,5.037694,124.0,123.0,21.128974,12.852966,2
92,92.0,-0.052448,0.025210,-0.903462,0.167045,0.0,94.0,30.056244,3.736873,-0.015544,...,6077.386712,4.974289,0.667494,-1.144814,5.040787,125.0,124.0,20.518337,13.585827,2


In [23]:
path_file = path_file.replace(r"M5\FAULT_M5_10", "")
os.chdir(path_file)
df_merged.to_csv('m5_fault10.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
